In [62]:
#1-1
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/5_1_1.csv')
#print(df.info())
#print(df.head(5))

df['conventional'] = df['conventional'].str.replace(r'[^a-zA-Z0-9가-힣]', '', regex=True)
#print(df.head())

df1 = df['conventional'].replace('', np.nan).astype(float)
#print(df1.info())
Q1 = df1.quantile(.25)
Q3 = df1.quantile(.75)
IQR = Q3-Q1
print(round(IQR, 0))

#1-2
cond1 = df['minority'] / df['poverty'] > 2
cond2 = df['city'] == 'state'
crime_mean = df.loc[cond1 & cond2, 'crime'].mean()
print(round(crime_mean,1))

#1-3
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/5_1_2.csv')
#print(df.info())
#print(df.head(2))

df['date'] = pd.to_datetime(dict(year = df.year, month = df.month, day = df.day))
result = df.loc[df['date'] >= '2016.09.01', 'actual'].mean()
print(round(result,0))

9.0
61.2
58.0


In [84]:
#2 (분류)
import pandas as pd
import numpy as np
train = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/5_2_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/5_2_test.csv')

# 2-0 (인포)
print(train.info())
print(test.info())
train['fractal_dimension_error'] = train['fractal_dimension_error'].fillna(train['fractal_dimension_error'].mean())
test['fractal_dimension_error'] = test['fractal_dimension_error'].fillna(test['fractal_dimension_error'].mean())
print(train.isna().sum())
print(test.isna().sum())

# 2-1 (데이터 분리)
train_X = train.drop('target', axis=1)
train_y = train['target']
test_X = test.drop('target', axis=1)
test_y = test['target']
test_id = test['ID']

# 2-2 (독-종 설정)
from sklearn.model_selection import train_test_split
train_X, valid_X, train_y, valid_y = train_test_split(train_X, train_y, test_size = 0.3, random_state=1)

# 2-3 (cat-num 분리)
cat_columns = train_X.select_dtypes('object').columns.to_list()
num_columns = train_X.select_dtypes('number').columns.to_list()

# 2-4 (전처리)
from sklearn.preprocessing import OneHotEncoder, StandardScaler
onehotencoder = OneHotEncoder(sparse_output = False, handle_unknown = 'ignore')
stdscaler = StandardScaler()

train_X_ohe = onehotencoder.fit_transform(train_X[cat_columns])
valid_X_ohe = onehotencoder.transform(valid_X[cat_columns])
test_X_ohe = onehotencoder.transform(test_X[cat_columns])

train_X_std = stdscaler.fit_transform(train_X[num_columns])
valid_X_std = stdscaler.transform(valid_X[num_columns])
test_X_std = stdscaler.transform(test_X[num_columns])

train_X_preprocessed = np.concatenate([train_X_ohe, train_X_std], axis=1)
valid_X_preprocessed = np.concatenate([valid_X_ohe, valid_X_std], axis=1)
test_X_preprocessed = np.concatenate([test_X_ohe, test_X_std], axis=1)

# 2-5 (모델적합)
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1)
rf.fit(train_X_preprocessed, train_y)

# 2-6 (검증 - 성능 확인)
from sklearn.metrics import roc_auc_score
pred_val = rf.predict_proba(valid_X_preprocessed)[:,1]
roc_auc_score(valid_y, pred_val)

# 2-7 (최종 예측)
test_pred = rf.predict_proba(test_X_preprocessed)[:,1]
test_pred = pd.DataFrame(test_pred, columns=['prob'])

# 2-8 (생성, 저장)
result = pd.concat([test_id, test_pred], axis=1)
result.to_csv('result.csv', index=False)
print(pd.read_csv('result.csv').head())

   level_0   ID  mean_radius  mean_texture  mean_perimeter  mean_area  \
0        0  249        11.52         14.93           73.87      406.3   
1        1   58        13.05         19.31           82.61      527.2   
2        2  476        14.20         20.53           92.41      618.4   
3        3  529        12.07         13.44           77.83      445.2   
4        4  422        11.61         16.02           75.46      408.2   

   mean_smoothness  mean_compactness  mean_concavity  mean_concave_points  \
0          0.10130           0.07808        0.043280             0.029290   
1          0.08060           0.03789        0.000692             0.004167   
2          0.08931           0.11080        0.050630             0.030580   
3          0.11000           0.09009        0.037810             0.027980   
4          0.10880           0.11680        0.070970             0.044970   

   ...  worst_texture  worst_perimeter  worst_area  worst_smoothness  \
0  ...          21.19     

In [128]:
#3-1
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/5_3_1.csv')
#print(df.info())
#print(df.head(2))

#3-1-1
study_hours_mean = df['study_hours'].mean()
study_hours_std = df['study_hours'].std()

exam_scores_mean = df['exam_scores'].mean()
exam_scores_std = df['exam_scores'].std()

print(round(study_hours_mean, 2))
print(round(study_hours_std, 2))
print(round(exam_scores_mean, 2))
print(round(exam_scores_std, 2))

#3-1-2
from scipy.stats import pearsonr

statistic, p_value = pearsonr(df['study_hours'], df['exam_scores'])
print(statistic.round(3))

#3-1-3
p_value = p_value.round(3)
if p_value < 0.05:
  result = '기각'
else:
  result = '채택'

print(result)

#3-2
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/YoungjinBD/data/main/exam/5_3_2.csv')
#print(df.info())
#print(df.head(2))

#3-2-1
df1 = df.groupby('campaign')['satisfaction_score'].mean()
df2 = df.groupby('campaign')['satisfaction_score'].std()
print(round(df1, 2))
print(round(df2, 2))



#3-2-2
from scipy.stats import f_oneway


df1 = df.loc[df['campaign']=='A', 'satisfaction_score']
df2 = df.loc[df['campaign']=='B', 'satisfaction_score']
df3 = df.loc[df['campaign']=='C', 'satisfaction_score']
statistic, p_value = f_oneway(df1, df2, df3)

print(round(statistic, 3))

#3-2-3 (마지막)
p_value = p_value.round(3)
if p_value < 0.05 :
  result = '기각'
else:
  rsult = '채택'
print(result)

5.28
2.27
74.79
8.76
-0.059
채택
campaign
A    71.41
B    74.79
C    82.57
Name: satisfaction_score, dtype: float64
campaign
A    11.37
B     8.76
C    10.43
Name: satisfaction_score, dtype: float64
15.606
기각
